# Chunking Strategies
In this notebook diffrent chunking mechanism is experimented on by making the other parameters constant in the simple rag pipeline to find the best chuncking mechanism.
The chunking mechanisms are:
- Small Chunking size for recursive character chunking 
- larger chunking size for recursive character chunking
- Semantic chunking mechanism

The idea behind this is by finding the optimal chunking we can improve the retrived context by making it more relevant. 

In [1]:
# Importing libraries
import sys
from dotenv import load_dotenv
import pandas as pd
sys.path.insert(1, '/home/jabez/week_11/Contract-Advisor-RAG')
load_dotenv()
sys.path.insert(1, '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/scripts')
import file_loader 
import pipelines 
import evaluation

/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Loading data
file_path = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/data/cnn_dailymail_3.0.0.csv'
data = file_loader.load_csv(file_path)

### RecursiveCharacterTextSplitter with 500 chunking size

In [3]:
# RecursiveCharacterTextSplitter with 500 chunking size
chunk_size= 500
chunk_overlap= 150
vectorstore_character = file_loader.character_text_splitter(data, chunk_size, chunk_overlap)

In [4]:
# Loading syntetic test data
syntetic_test_data =pd.read_csv('/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data.csv')

In [5]:
retriver = vectorstore_character.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [6]:
# Adding answer to test data from simple pipeline
syntetic_test_data_with_answer = evaluation.adding_answer_to_testdata(syntetic_test_data, pipelines.simple_pipeline, vectorstore_character, retriver)

/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [7]:
# Evaluating the test data from simple pipeline
simple_rag_evaluation_result = evaluation.ragas_evaluator(syntetic_test_data_with_answer)

Evaluating: 100%|██████████| 80/80 [00:37<00:00,  2.13it/s]


In [8]:
# Evaluation mean
result = evaluation.evaluation_mean(simple_rag_evaluation_result)

context_precision: 89.22%, faithfulness: 81.23%, answer_relevancy: 90.14%, context_recall: 79.33%


In [9]:
simple_rag_evaluation_result[['context_precision','faithfulness','answer_relevancy','context_recall']]

,context_precision,faithfulness,answer_relevancy,context_recall
0,1.000000,0.600000,0.882616,1.000000
1,1.000000,1.000000,0.970551,1.000000
2,1.000000,0.909091,0.999299,1.000000
3,1.000000,0.500000,1.000000,0.500000
4,0.000000,0.000000,0.775418,0.000000
5,1.000000,0.800000,1.000000,1.000000
6,1.000000,1.000000,0.955961,1.000000
7,0.926667,0.875000,0.984427,0.833333
8,1.000000,1.000000,1.000000,1.000000
9,0.916667,1.000000,1.000000,1.000000


### RecursiveCharacterTextSplitter with 1000 chunking size

In [10]:
# RecursiveCharacterTextSplitter with 1000 chunking size
chunk_size= 1000
chunk_overlap= 250
vectorstore_character_large = file_loader.character_text_splitter(data, chunk_size, chunk_overlap)

In [11]:
retriver_large = vectorstore_character_large.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [12]:
# Adding answer to test data from simple pipeline
syntetic_test_with_answer_large = evaluation.adding_answer_to_testdata(syntetic_test_data, pipelines.simple_pipeline, vectorstore_character_large, retriver_large)

In [13]:
# Evaluating the test data from simple pipeline
simple_rag_large_evaluation_result = evaluation.ragas_evaluator(syntetic_test_with_answer_large)

Evaluating: 100%|██████████| 80/80 [00:40<00:00,  2.00it/s]


In [16]:
# Evaluation mean
result = evaluation.evaluation_mean(simple_rag_evaluation_result)

context_precision: 89.22%, faithfulness: 81.23%, answer_relevancy: 90.14%, context_recall: 79.33%


In [15]:
# Evaluation mean
result = evaluation.evaluation_mean(simple_rag_large_evaluation_result)

context_precision: 91.25%, faithfulness: 84.36%, answer_relevancy: 90.81%, context_recall: 82.92%


#### Semantic Chunking

In [15]:
# Setting semantic text splitter
vectorstore_semantic = file_loader.semantic_text_splitter(data)

In [17]:
# Setting retriever for semantic based chuncking
retriver_semantic = vectorstore_semantic.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [18]:
# Adding answer to test data from simple pipeline
syntetic_test_data_with_answer = evaluation.adding_answer_to_testdata(syntetic_test_data, pipelines.simple_pipeline, vectorstore_semantic, retriver_semantic)

In [21]:
# Evaluating the test data from simple pipeline
simple_rag_evaluation_result = evaluation.ragas_evaluator(syntetic_test_data_with_answer)

Evaluating: 100%|██████████| 80/80 [00:55<00:00,  1.44it/s]


In [22]:
# Evaluation mean
result = evaluation.evaluation_mean(simple_rag_evaluation_result)

context_precision: 90.02%, faithfulness: 80.08%, answer_relevancy: 77.02%, context_recall: 82.92%


### Results:
- Recursive Character Chunking with 500 chunk size:
context_precision: 94.83%, faithfulness: 92.05%, answer_relevancy: 86.31%, context_recall: 90.5%
- Recursive Character Chunking with 1000 chunk size:
context_precision: 94.17%, faithfulness: 91.71%, answer_relevancy: 81.33%, context_recall: 76.83%
- Semantic chunking result:
context_precision: 90.02%, faithfulness: 80.08%, answer_relevancy: 77.02%, context_recall: 82.92%
